# <span style="font-family: Arial, sans-serif; color:#97f788">xbooster</span>
## <span style="font-family: Arial, sans-serif; color:navyblue">SQL deployment</span>

<span style="font-family: Arial, sans-serif; color:navyblue">Repo: <a href="https://github.com/xRiskLab/xBooster" title="GitHub link">https://github.com/xRiskLab/xBooster</a></span>

In [1]:
import pickle
import pandas as pd

with open("models/scorecard_constructor.pkl", "rb") as file:
    scorecard_constructor = pickle.load(file)

X = pd.read_parquet("data/X_test.parquet")

In [2]:
sql_query = scorecard_constructor.sql_query
print(sql_query)

WITH scorecard AS
(
    SELECT *,
    CASE WHEN ((account_never_delinq_percent < 98) 
          AND (revolving_utilization_of_unsecured_lines >= 0.60930562 OR revolving_utilization_of_unsecured_lines IS NULL) 
          AND (external_risk_estimate < 70)) 
     THEN 0
          WHEN ((account_never_delinq_percent < 98) 
          AND (revolving_utilization_of_unsecured_lines >= 0.60930562 OR revolving_utilization_of_unsecured_lines IS NULL) 
          AND (external_risk_estimate >= 70 OR external_risk_estimate IS NULL)) 
     THEN 20
          WHEN ((account_never_delinq_percent < 98) 
          AND (revolving_utilization_of_unsecured_lines < 0.60930562) 
          AND (external_risk_estimate < 69)) 
     THEN 22
          WHEN ((account_never_delinq_percent < 98) 
          AND (revolving_utilization_of_unsecured_lines < 0.60930562) 
          AND (external_risk_estimate >= 69 OR external_risk_estimate IS NULL)) 
     THEN 33
          WHEN ((account_never_delinq_percent >= 98 OR accou

### <span style="font-family: Arial, sans-serif; color: navyblue; font-size: 25px;">DuckDB</span>

In this step, we create a DuckDB database and load the data into it.

After this we can test the functionality of `convert_to_sql` function.

In [3]:
import duckdb
import numpy as np

# We need to use np.float32 to reproduce predict_score
my_df = X.astype(np.float32).copy()

duckdb.sql("DROP TABLE if exists my_table")
duckdb.sql("CREATE TABLE my_table AS SELECT * FROM my_df")
duckdb.sql("SELECT * FROM my_table LIMIT 5")

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┐
│ external_risk_esti…  │ revolving_utilizat…  │ account_never_deli…  │ … │ num_total_cc_accou…  │ average_months_in_…  │
│        float         │        float         │        float         │   │        float         │        float         │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┤
│                 74.0 │            0.9999999 │                100.0 │ … │                 14.0 │                103.0 │
│                 72.0 │          0.119103305 │                 89.7 │ … │                 17.0 │                 48.0 │
│                 66.0 │          0.067186296 │                100.0 │ … │                  2.0 │                121.0 │
│                 75.0 │            0.9999999 │                 77.3 │ … │                 15.0 │                 69.0 │
│                 85.0 │        

Let's check if can get the same output from the scorecard using `predict_score`.

In [4]:
scores = duckdb.sql(f"{sql_query}").to_df()
scores_py = scorecard_constructor.predict_score(X)
scores["score"].describe() == scores_py.describe()

count    True
mean     True
std      True
min      True
25%      True
50%      True
75%      True
max      True
dtype: bool